In [1]:
import os
import json
import requests
from tqdm import tqdm
from collections import defaultdict, Counter
from sparc.client import SparcClient

# === Setup ===
client = SparcClient(connect=False, config_file='config.ini')

# === Step 1: Run Elasticsearch-style query to get a known ID range ===
ids = list(range(0, 1001))
id_strings = [f'"{i}"' for i in ids]
id_list_str = ", ".join(id_strings)

body = f'''
{{
  "size": 1000,
  "query": {{
    "terms": {{
      "_id": [ {id_list_str} ]
    }}
  }}
}}
'''
body_json = json.loads(body)
response = client.metadata.search_datasets(body_json)

# === Step 2: Filter and categorize by type ===
type_counter = Counter()
type_id_map = defaultdict(list)

for d in response["hits"]["hits"]:
    try:
        type_name = d["_source"]["item"]["types"][0]["name"]
    except (KeyError, IndexError, TypeError):
        type_name = "<invalid or missing type>"

    type_counter[type_name] += 1
    dataset_id = d.get("_id", "<no id>")
    type_id_map[type_name].append(dataset_id)

# === Step 3: Show summary ===
print("Dataset type counts:")
for t, count in type_counter.items():
    print(f"{t}: {count}")

print("\nProcessing only type = 'dataset'")
dataset_ids = type_id_map["dataset"]
print(f"Found {len(dataset_ids)} datasets.\n")

# === Step 4: Process only datasets and collect extensions ===
dataset_paths = []
non_dataset_count = 0
error_count = 0
extension_counter = Counter()

for dataset_id in tqdm(dataset_ids, desc="Processing datasets"):
    # === Fetch metadata ===
    metadata_url = f"https://api.pennsieve.io/discover/datasets/{dataset_id}/versions/1/metadata"
    try:
        meta_response = requests.get(metadata_url)
        meta_response.raise_for_status()
        metadata = meta_response.json()
    except Exception as e:
        tqdm.write(f"[ERROR] Metadata fetch failed for dataset {dataset_id}: {e}")
        error_count += 1
        continue

    # === Extract 'files/primary/' paths and extensions ===
    primary_paths = [
        f.get("path") for f in metadata.get("files", [])
        if f.get("path", "").startswith("files/primary/")
    ]

    if primary_paths:
        dataset_paths.append((dataset_id, primary_paths))
        for path in primary_paths:
            _, ext = os.path.splitext(path)
            ext = ext.lower() if ext else "<no extension>"
            extension_counter[ext] += 1

        tqdm.write(f"  Primary file paths for dataset {dataset_id}: {primary_paths}")
    else:
        tqdm.write(f"  No files under 'files/primary/' for dataset {dataset_id}.")
        non_dataset_count += 1

# === Final Summary ===
tqdm.write("\n=== Summary ===")
tqdm.write(f"Total datasets queried: {len(dataset_ids)}")
tqdm.write(f"Metadata errors: {error_count}")
tqdm.write(f"Datasets without primary files: {non_dataset_count}")
tqdm.write(f"Datasets with primary files: {len(dataset_paths)}")

/Users/admin/miniconda3/envs/sparc_vns/lib/python3.10/site-packages/sparc/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)


Dataset type counts:
dataset: 240
scaffold: 42
computational model: 47
<invalid or missing type>: 7
device: 1

Processing only type = 'dataset'
Found 240 datasets.



Processing datasets:   0%|          | 1/240 [00:01<05:30,  1.38s/it]

  Primary file paths for dataset 120: ['files/primary/sub-cat9/sam-1_cat9/DICOM 3D CT pre surgical bladder/DAMASER.XA._.0001.0001.2020.02.13.10.29.10.109375.7722025.IMA', 'files/primary/sub-cat9/sam-1_cat9/DICOM 3D CT pre surgical bladder/DAMASER.XA._.0001.0002.2020.02.13.10.29.10.109375.7722041.IMA', 'files/primary/sub-cat9/sam-1_cat9/DICOM 3D CT pre surgical bladder/DAMASER.XA._.0001.0003.2020.02.13.10.29.10.109375.7722057.IMA', 'files/primary/sub-cat9/sam-1_cat9/DICOM 3D CT pre surgical bladder/DAMASER.XA._.0001.0004.2020.02.13.10.29.10.109375.7722073.IMA', 'files/primary/sub-cat9/sam-1_cat9/DICOM 3D CT pre surgical bladder/DAMASER.XA._.0001.0005.2020.02.13.10.29.10.109375.7722089.IMA', 'files/primary/sub-cat9/sam-1_cat9/DICOM 3D CT pre surgical bladder/DAMASER.XA._.0001.0006.2020.02.13.10.29.10.109375.7722105.IMA', 'files/primary/sub-cat9/sam-1_cat9/DICOM 3D CT pre surgical bladder/DAMASER.XA._.0001.0007.2020.02.13.10.29.10.109375.7722121.IMA', 'files/primary/sub-cat9/sam-1_cat9/DI

Processing datasets:   1%|          | 2/240 [00:02<04:06,  1.04s/it]

...

  Primary file paths for dataset 125: ['files/primary/Tertiary_Plexus_density_from_immunohistochemistry/pool-Tertiary_Plexus_density_from_immunohistochemistry/LMMNs_Tertiary plexus fibre counts_SBv4.xlsx', 'files/primary/Tertiary_Plexus_density_from_immunohistochemistry/pool-Tertiary_Plexus_density_from_immunohistochemistry/README.txt', 'files/primary/LMMNs_Tenia NOS VIP DiI/README.txt', 'files/primary/LMMNs_Tenia NOS VIP DiI MICROGRAPHS/README.txt', 'files/primary/LMMNs_Inter-taenial ChAT NOS DiI MICROGRAPHS/README.txt', 'files/primary/LMMNs_Inter-taenial ChAT NOS DiI/README.txt', 'files/primary/LMMNs_Tenia ChAT NOS DiI/README.txt', 'files/primary/LMMNs_Tenia ChAT NOS DiI MICROGRAPHS/README.txt', 'files/primary/LMMNs_Inter-taenial ChAT NOS DiI/sub-H2100/sam-H2100/SPARC_H2100A_LM_NOS%26ChAT_DiI.xlsx', 'files/primary/LMMNs_Inter-taenial ChAT NOS DiI MICROGRAPHS/sub-H2100/sam-H2100/SPARC_H2100A_LM_MICROGRAPHS/SPARC_H2100A_LM_X-0.303Y-4.213_ChAT', 'files/primary/LMMNs_Inter-taenial ChAT N

Processing datasets:  29%|██▉       | 69/240 [01:04<02:59,  1.05s/it]

  Primary file paths for dataset 126: ['files/primary/Optogenetic_activation/sub-nNOS_Male_Subject_1/sam-190829/19 0829.acq', 'files/primary/Optogenetic_activation/sub-nNOS_Male_Subject_2/sam-190917/19 0917.acq', 'files/primary/Optogenetic_activation/sub-nNOS_Male_Subject_3/sam-191016/19 1016.acq', 'files/primary/Optogenetic_activation/sub-nNOS_Female_Subject_1/sam-191018/19 1018.acq', 'files/primary/Optogenetic_activation/sub-nNOS_Female_Subject_2/sam-200303/20 0303.acq', 'files/primary/Optogenetic_activation/sub-ChAT_Male_Subject_1/sam-200309/20 0309.acq', 'files/primary/Optogenetic_activation/sub-nNOS_Female_Subject_3/sam-200324/20 0324.acq', 'files/primary/Optogenetic_activation/sub-ChAT_Male_Subject_2/sam-200515/20 0515.acq', 'files/primary/Optogenetic_activation/sub-ChAT_Male_Subject_3/sam-200518/20 0518.acq', 'files/primary/Optogenetic_activation/sub-ChAT_Female_Subject_1/sam-200519/20 0519.acq', 'files/primary/Optogenetic_activation/sub-ChAT_Female_Subject_2/sam-200520/20 0520.

Processing datasets:  29%|██▉       | 69/240 [01:05<02:41,  1.06it/s]


KeyboardInterrupt: 

In [ ]:
# === Extension Summary ===
tqdm.write("\nAvailable file extensions in 'files/primary/':")
for ext, count in sorted(extension_counter.items(), key=lambda x: -x[1]):
    if count >= 100:  # Only show extensions with significant counts
        tqdm.write(f"{ext}: {count}")

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter, defaultdict
import seaborn as sns
from matplotlib.ticker import ScalarFormatter

# Use seaborn for cleaner minimal plots
sns.set_theme(style="white")

# Ensure output directory exists
output_dir = './stats_figures'
os.makedirs(output_dir, exist_ok=True)

# === Example input structure ===
# dataset_paths = [('ds1', ['file1.mat', 'image1.tif']), ('ds2', ['notes.pdf', 'data.csv'])]

# Map extensions to categories, merging Tabular and Structured into Docs
modality_lookup = {
    # Imaging formats
    ".tif": "Imaging", ".tiff": "Imaging", ".czi": "Imaging", ".nd2": "Imaging", ".lsm": "Imaging",
    ".jpx": "Imaging", ".svs": "Imaging", ".ims": "Imaging", ".png": "Imaging", ".jpg": "Imaging",
    ".jpeg": "Imaging", ".bmp": "Imaging", ".vsi": "Imaging", ".jp2": "Imaging", ".roi": "Imaging",
    ".dm3": "Imaging", ".pxp": "Imaging", ".ipf": "Imaging", ".lif": "Imaging", ".ima": "Imaging",
    ".mrxs": "Imaging", ".obj": "Imaging", ".avi": "Imaging", ".exf": "Imaging", ".cxd": "Imaging",

    # Time Series formats
    ".mat": "Time Series", ".smr": "Time Series", ".csv": "Time Series",
    ".adicht": "Time Series", ".hdf5": "Time Series", ".h5": "Time Series", ".ets": "Time Series",
    ".abf": "Time Series", ".rhd": "Time Series", ".nev": "Time Series", ".ns5": "Time Series",
    ".ns2": "Time Series", ".ns1": "Time Series", ".smrx": "Time Series", ".wav": "Time Series",
    ".acq": "Time Series", ".tbk": "Time Series", ".tdx": "Time Series", ".tev": "Time Series",
    ".tin": "Time Series", ".tnt": "Time Series", ".tsq": "Time Series", ".eeg": "Time Series",
    ".vmrk": "Time Series", ".vhdr": "Time Series", ".sev": "Time Series", ".sam": "Time Series",
    ".pss": "Time Series", ".psmethod": "Time Series",

    # Documentation formats
    ".pdf": "Docs", ".docx": "Docs", ".doc": "Docs", ".txt": "Docs",
    ".xlsx": "Docs", ".xls": "Docs", ".tsv": "Docs", ".json": "Docs", 
    ".xml": "Docs", ".db": "Docs", ".xfg": "Docs",
    
    # Other formats
    ".inf": "Other", ".zip": "Other", "": "Other", "(no ext)": "Other",
    ".s2r": "Other", ".ini": "Other", ".cmgui": "Other",
    ".mp4": "Other", ".gz": "Other", ".xlsm": "Other",
    ".db3": "Other", ".ccf": "Other", ".ex": "Other",
    ".conf": "Other", ".rdf": "Other", ".vtk": "Other", ".proj": "Other", ".pnp": "Other",
    ".hoc": "Other", ".fig	": "Other", ".dat": "Other"
}

# === Summary structures ===
summary = {}
modality_extension_counts = defaultdict(Counter)
dataset_category = defaultdict(set)
datasets_with_timeseries = set()

# === Process the dataset file paths ===
for dataset_id, paths in dataset_paths:
    has_ts = False
    has_img = False
    has_docs = False
    for p in paths:
        ext = os.path.splitext(p)[1].lower() or '(no ext)'
        category = modality_lookup.get(ext, 'Other')
        summary[ext] = summary.get(ext, 0) + 1
        modality_extension_counts[category][ext] += 1
        if category == 'Time Series':
            has_ts = True
        elif category == 'Imaging':
            has_img = True
        elif category == 'Documentation':
            has_docs = True
        dataset_category[category].add(dataset_id)
    if has_ts:
        datasets_with_timeseries.add(dataset_id)
    elif has_img:
        dataset_category['Imaging'].add(dataset_id)
    elif has_docs:
        dataset_category['Documentation'].add(dataset_id)
    else:
        dataset_category['Other'].add(dataset_id)


# === Create summaries ===
summary_df = pd.DataFrame.from_dict(summary, orient='index', columns=['File Count'])
summary_df.index.name = 'Extension'
summary_df['Category'] = summary_df.index.map(lambda x: modality_lookup.get(x, 'Other'))
summary_df = summary_df.sort_values('File Count', ascending=False)

category_summary = (
    summary_df[summary_df['Category'] != 'Unknown']
    .groupby('Category')['File Count']
    .sum()
    .sort_values(ascending=False)
)

dataset_series = pd.Series({cat: len(ids) for cat, ids in dataset_category.items() if cat != 'Unknown'}).sort_values(ascending=False)
ts_series = pd.Series(modality_extension_counts['Time Series']).sort_values(ascending=False)
img_series = pd.Series(modality_extension_counts['Imaging']).sort_values(ascending=False)

# === Plot configuration ===
TICK_FS = 10
LABEL_FS = 12

# For a half 16:9 slide (6 x 8 inches)
fig, axes = plt.subplots(4, 1, figsize=(6, 8))

# Y-axis label texts for each subplot
y_labels = ['Files', 'Datasets', 'Time Series Files', 'Imaging Files']
i = 0
for ax, data, ylab in zip(axes, [category_summary, dataset_series, ts_series, img_series], y_labels):
    if i<=1:
        #filter out docs
        data = data[data.index != 'Docs']
    ax.bar(data.index, data.values, color=sns.color_palette('plasma', len(data)))
    ax.tick_params(axis='x', labelsize=TICK_FS)
    ax.tick_params(axis='y', labelsize=TICK_FS)
    ax.set_ylabel(ylab, fontsize=LABEL_FS)
    # Scientific notation
    if i==0:
        ax.ticklabel_format(style='scientific', axis='y', scilimits=(0,0), useMathText=True)
    # Match offset text size
    offset = ax.yaxis.get_offset_text()
    offset.set_fontsize(TICK_FS)
    i += 1

# Rotate x-ticks for last two
for ax in axes[2:]:
    ax.tick_params(axis='x', rotation=45)

sns.despine()
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'combined_4x1_summary_minimal_sorted.svg'), format='svg')
plt.show()


# === Word Cloud: File Extension Frequencies ===
from wordcloud import WordCloud

# Filter extension_counter to only include counts ≥ 100
wc_freq = {
    ext.lstrip('.') or 'no_ext': count
    for ext, count in extension_counter.items()
    if count >= 1
}

# Generate word cloud
graph = WordCloud(width=1600, height=800, background_color='white').generate_from_frequencies(wc_freq)

# Plot word cloud
plt.figure(figsize=(5, 2.5))
plt.imshow(graph)
plt.axis('off')
plt.tight_layout()
# Save the word cloud
plt.savefig(os.path.join(output_dir, 'file_extensions_wordcloud.svg'), format='svg')
plt.show()

# print ids per category
print("\nDataset IDs per category:")
print("=== Imaging Datasets ===")
print(f"({len(dataset_category['Imaging'])}): {', '.join(map(str, dataset_category['Imaging']))}")
print("\n=== Time Series Datasets ===")
print(f"({len(dataset_category['Time Series'])}): {', '.join(map(str, dataset_category['Time Series']))}")
print("\n=== Documentation Datasets ===")
print(f"({len(dataset_category['Documentation'])}): {', '.join(map(str, dataset_category['Documentation']))}")
print("\n=== Other Datasets ===")
print(f"({len(dataset_category['Other'])}): {', '.join(map(str, dataset_category['Other']))}")

# print totla number of file formats
print(f"\nTotal number of unique file formats: {len(modality_lookup)}")

### Prepare Figures fo GitHub Readme

In [ ]:
TICK_FS, LABEL_FS = 10, 12
fig, axes = plt.subplots(2, 2, figsize=(12, 4))
axes = axes.flatten()                       # ★ flatten fixes the 2×2 bug ★

plots_and_labels = [
    (category_summary[category_summary.index != "Docs"], "Files"),
    (dataset_series[dataset_series.index != "Docs"], "Datasets"),
    (ts_series, "Time Series Files"),
    (img_series, "Imaging Files"),
]

for ax, (data, ylab) in zip(axes, plots_and_labels):
    ax.bar(data.index, data.values, color=sns.color_palette("plasma", len(data)))
    ax.set_ylabel(ylab, fontsize=LABEL_FS)
    ax.tick_params(axis="x", labelsize=TICK_FS)
    ax.tick_params(axis="y", labelsize=TICK_FS)
    if ylab == "Files":
        ax.ticklabel_format(style="scientific", axis="y", scilimits=(0, 0), useMathText=True)
        ax.yaxis.get_offset_text().set_fontsize(TICK_FS)

for ax in axes[2:]:
    ax.tick_params(axis="x", rotation=45)

sns.despine()
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "combined_2x2_summary.png"), format="png")
plt.show()

# ── Word cloud of extensions ──────────────────────────────────────────────────
wc_freq = {
    ext.lstrip(".") or "no_ext": count
    for ext, count in extension_counter.items()
    if count >= 1
}

plt.figure(figsize=(5, 2.5))
WordCloud(width=1600, height=800, background_color="white")\
    .generate_from_frequencies(wc_freq)\
    .to_image()
plt.imshow(WordCloud(width=1600, height=800, background_color="white").generate_from_frequencies(wc_freq))
plt.axis("off")
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "file_extensions_wordcloud.svg"), format="svg")
plt.show()

# ── Console summaries ────────────────────────────────────────────────────────
print("\nDataset IDs per category")
for cat in ["Imaging", "Time Series", "Docs", "Other"]:
    ids = sorted(dataset_category[cat])
    print(f"=== {cat} ({len(ids)}) ===")
    print(", ".join(map(str, ids)) or "—")

print(f"\nTotal number of unique file formats: {len(modality_lookup)}")
